# Import development libraries

In [13]:
import bw2data as bd
import bw2calc as bc
import bw_processing as bwp
import numpy as np

In [14]:
bd.projects.set_current("ecoinvent 3.7.1")

In [15]:
tox = ('ReCiPe Midpoint (E) V1.13', 'human toxicity', 'HTPinf')

In [16]:
[(o.id, o.key) for o in bd.Database("ecoinvent 3.7.1") if o['name'] == 'transport, passenger car, electric']

[(7301, ('ecoinvent 3.7.1', '6f67d7bb34034ed6aef5f33536ae7781'))]

# Pre-analyze the database and identify market-like activities

In [5]:
from bw2data.backends.schema import ActivityDataset as AD, ExchangeDataset as ED

In [6]:
def get_count_similar_exchanges(act):
    return (
        ED.select()
            .join(AD, on=((ED.input_database == AD.database) & (ED.input_code == AD.code)))
            .where((ED.output_code == act['code']) &
                   (ED.input_code != act['code']) &
                   (ED.output_database == act['database']) &
                   (AD.product == act['reference product'])
                  )
            .count()
    )

In [7]:
markets = {obj.id for obj in bd.Database('ecoinvent 3.7.1') if get_count_similar_exchanges(obj) > 1}

In [8]:
len(markets)

3587

In [9]:
[x for x in bd.Database('ecoinvent 3.7.1') 
     if 'market' not in x['name'].lower() 
     and 'electricity' not in x['name'].lower() 
     and x.id in markets]

['natural gas, high pressure, import from NL' (cubic meter, CH, None),
 'fibre production, viscose' (kilogram, GLO, None),
 'natural gas, high pressure, import from NL' (cubic meter, IT, None),
 'primary zinc production from concentrate' (kilogram, RoW, None),
 'hard coal, import from ZA' (kilogram, IN, None),
 'folding boxboard carton production' (kilogram, RoW, None),
 'natural gas, high pressure, import from NL' (cubic meter, DE, None),
 'tap water production, ultrafiltration treatment' (kilogram, RoW, None),
 'natural gas, high pressure, import from DZ' (cubic meter, IT, None),
 'zinc mine operation' (kilogram, GLO, None),
 'natural gas, high pressure, import from DZ' (cubic meter, ES, None),
 'sulfate pulp production, from eucalyptus, bleached' (kilowatt hour, RoW, None),
 'natural gas, high pressure, import from NL' (cubic meter, DK, None),
 'particleboard production, uncoated, average glue mix' (megajoule, RoW, None),
 'natural gas, high pressure, import from NL' (cubic meter, F

Ideally, we would filter this down to see which markets present meaningful differences.

# Graph traversal of this LCA

Electric car, human toxicity

https://2.docs.brightway.dev/lca.html#graph-traversal

In [17]:
fu, data_objs, _ = bd.prepare_lca_inputs({('ecoinvent 3.7.1', '6f67d7bb34034ed6aef5f33536ae7781'): 1}, method=tox)

In [18]:
lca = bc.LCA(fu, data_objs=data_objs)

In [19]:
lca.lci()
lca.lcia()

In [20]:
lca.score

7.987990671102498

In [21]:
gt = bc.GraphTraversal()
gt_results = gt.calculate(lca, cutoff=0.005, max_calc=1e5)

In [22]:
gt_results['counter']

4005

In [23]:
len(gt_results['nodes']), len(gt_results['edges'])

(340, 745)

In [24]:
gt_results['edges'][100]

{'to': 11269,
 'from': 1067,
 'amount': 5.717765640811612e-09,
 'exc_amount': 0.09144452214241028,
 'impact': 0.07269294239417574}

In [25]:
gt_results['nodes'][-1]

{'amount': 1, 'cum': 7.987990671102498, 'ind': 7.987990671102498e-06}

# Find most important markets in this graph

In [26]:
markets_as_indices = {lca.dicts.activity[i] for i in markets}

In [27]:
market_importance = sorted([(v['cum'], k) for k, v in gt_results['nodes'].items() if k in markets_as_indices], reverse=True)
len(market_importance)

104

In [28]:
def market_inputs(act):
    return [exc.input for exc in act.technosphere() if exc.input['reference product'] == exc.output['reference product']]

In [29]:
def sum_market_inputs(act):
    return sum([exc['amount'] for exc in act.technosphere() if exc.input['reference product'] == exc.output['reference product']])

In [30]:
[(
    x, 
    y, 
    sum_market_inputs(bd.get_activity(lca.dicts.activity.reversed[y])),
    len(market_inputs(bd.get_activity(lca.dicts.activity.reversed[y]))),
    bd.get_activity(lca.dicts.activity.reversed[y]),
) for x, y in market_importance[:10]]

[(3.2523921131737827,
  7589,
  0.9999999999999993,
  19,
  'market for copper, cathode' (kilogram, GLO, None)),
 (2.508005740187961,
  495,
  0.9999999999999996,
  8,
  'market group for electricity, low voltage' (kilowatt hour, GLO, None)),
 (2.145363505188869,
  13234,
  1.0000000000000004,
  2,
  'market for battery cell, Li-ion' (kilogram, GLO, None)),
 (2.0428948061282557,
  17773,
  1.0000000000000002,
  7,
  'market for copper, anode' (kilogram, GLO, None)),
 (1.9902980928882927,
  7915,
  1.0000000000000004,
  2,
  'market for anode, graphite, for lithium-ion battery' (kilogram, GLO, None)),
 (1.3751037128882988,
  2176,
  1.000000000000001,
  16,
  'market for copper concentrate, sulfide ore' (kilogram, GLO, None)),
 (1.2392523987707136,
  15892,
  1.0000000000000002,
  40,
  'market group for electricity, low voltage' (kilowatt hour, RAS, None)),
 (1.2113578656449906,
  11269,
  0.9999999999999999,
  17,
  'market for gold' (kilogram, GLO, None)),
 (0.5685996213602118,
  756

# Creating possibility arrays

In [28]:
def create_replacement_array_dp(activities):
    modified = bwp.create_datapackage(combinatorial=True)
    
    for activity in activities:
        possibles = market_inputs(activity)
        data = np.diag(np.ones(len(possibles))) * -1 / sum_market_inputs(activity)
        indices = np.zeros(len(possibles), dtype=bwp.INDICES_DTYPE)
    
        for index, obj in enumerate(possibles):
            indices[index] = (obj.id, activity.id)
                
        modified.add_persistent_array(
            matrix="technosphere_matrix",
            indices_array=indices,
            name=f"Possibilities for {activity}",
            data_array=data,
        )
    return modified

In [29]:
replacements = create_replacement_array_dp(
    [bd.get_activity(lca.dicts.activity.reversed[y]) 
     for _, y in market_importance[:3]]
)

In [30]:
lca = bc.LCA(
    fu, 
    data_objs=(
        data_objs + 
        [replacements]
    ),
    use_arrays=True
)

In [31]:
lca.lci()
lca.lcia()

In [32]:
while True:
    try:
        print(lca.score)
        next(lca)
    except StopIteration:
        break

11.01581157934838
2.921496096865127
4.892383606318661
4.696088936385785
4.9973993493367725
4.737278039002138
3.2096592191918054
4.954830358643444
17.941941190688087
9.799877906655114
11.826597960847552
11.624324182793492
11.91589394959018
11.668296945502446
10.102497794587574
11.84454063915459
11.057995522058476
2.9634155651259415
4.934636874614836
4.7383064146118725
5.039556486500309
4.779512344527402
3.2516658528957625
4.99681224029698
12.45541495636949
4.3511938453308385
6.333681745631473
6.136144758655459
6.435429843176849
6.177912357503262
4.642361190447372
6.386878041643239
12.662660981543459
4.55693426778861
6.541111154802263
6.343393420682683
6.642389979650333
6.385244688818056
4.84853680279185
6.59297665394694
11.057064109605648
2.962492567027934
4.933705895088073
4.7373762865302895
5.038627591500575
4.7785818325098735
3.2507408437387673
4.995887220495532
11.358111277472048
3.2614602914278095
5.2351013693021775
5.038511779680031
5.339339860096596
5.079838338170874
3.5503370739